### HKN DeCal Lab 11
## Controls & Embedded Systems Lab
By Alex Kristofferson, Simon Guo, & Bryan Ngo

## Part A. Understanding Feedback Control


### Lecture-Recap & Motivation
Skip this part if you are comfortable with the lecture content.

In the lecture, we talked about embedded systems: systems that process input from the environment and exert some output to achieve specific goals. The system architecture can be summarized in the diagram below.

![Embedded System Control Flow](Embedded-Systems.jpg)

Since we are operating in a physical world, we must interface with analog signals. 
1. Sensors typically read in analog signals (e.g., voltage measurement on thermistor that depends on temperature). We must convert to digital for embedded systems to understand.
2. When we want to use an actuator to do something (e.g., changing motor speed), we must give the actuator an analog signal (e.g., how many volts for the motor), and that has to be converted from the digital signals.

Because of the need for conversion between the digital and analog world, and the physical world is complicated and not always work in our wish, we have some challenges:
1. Sensor reading might be noisy and fluctuating; we can use **filtering** to filter out some noise and use Fourier transform to focus on specific frequencies.
2. When we output to the actuator, let's say we want the motor to increase to 60 rpm; it might not achieve that precisely since many factors play in the physical world. We must continuously adjust, and that's the topic of this lab, **controls**.

### What are controls and controllers?

Now that we learned about why we need control: to let our actuator perform in the way we want. 
There are mostly two types of control:

**Open-Loop Control** - we calculate the output and give that to the actuator. We just let it run, and we don't know what the effect is or if it is performing at the desired situation we want. Think about powering a fan; you just need it to spin rather than checking if it is at a certain RPM.

**Closed-Loop Feedback Control** - As you can see, open-loop control does not solve the issue we mentioned before. Thus, we need a way to adjust the output to the actuator based on the system's situation. **State** is a word that describes the state or specific information about a system at a particular time. 

For example, you are heating a room. Your state will be the current temperature of the room. You want to reach the desired temperature. If you can measure the current state, you can adjust the power of the heater. If your room is still too cold, you increase the power. If the current temperature is closed to the desired temperature, you can reduce power. If it is over the temperature, you can stop the heater for some time. 

You just saw an example of a closed-loop feedback system, where the signal to the controller/actuator is based on the state of the system. You are getting **feedback** from the state of the system to adjust your control signals further. 

To summarize, you continuously apply a signal to the actuator; the actuator does its work. You check the system's updated state, recalculate the new signal needed to the actuator, and repeat. This is like a cycle, so we call this **closed-loop**.

There are **many types** of controllers that help us achieve this. They differ in how feedback is weighted or how the state is translated into the new control signal. To name a few:
* Proportional–Integral–Derivative (PID) Controller
* Linear-Quadratic Regulator (LQR)
* Nonlinear Controllers
* Robust Control

In this lab, we will focus on PID controllers and see how they can be used to stabilize systems that are generally very unstable. Very powerful indeed!

## Part B. Proportional–Integral–Derivative (PID) Controller

Let's take a look at what PID Controller is and how we can use them.
On a high level, the controller calculates an error value, called $e(t)$, representing the difference between the current state and desired state. Having that error value or difference in mind, we can apply a correction input into the system (output signal to the actuator) based on the combined effect of **proportional, integral, and derivative terms**. That is why it is called PID.

Here is a diagram that illustrates the process we just described.

![PID Control Flow](PID.png)

Notice a loop in the diagram. With this closed-loop feedback, our current state can be used to adjust our next control signal, eventually leading the system to **stabilize** around the desired state.

The equation is shown below; notice the three terms that apply proportional, integral, and derivative operations on the $e(t)$ value.

$$u(t)=K_{\text{p}}e(t)+K_{\text{i}}\int _{0}^{t}e(t')\,dt'+K_{\text{d}}{\frac {de(t)}{dt}}$$

$u(t)$ represents the control signal we calculated to apply to the actuator, based on our current state.

As mentioned, there are three terms we are using to calculate that $u(t)$. Three $K$ values respectively tune the three methods (proportional, integral, and derivative); you can think about them as weights or coefficients that scale each operation's effect.

These $K$ factors are called tuning constants, and they are unique to each application. Determining these factors is an area worth investigating by itself. The unique set of values chosen for these $K$ values create a **controller** from this control method!

PID controllers are widely used in cyber-physical systems, such as a car! Try out [this car PID controller demo](https://sites.google.com/site/fpgaandco/pid) and see how you can adjust the 3 $K$ tuning constants to build a PID controller to make the car stable!

## Part C. Applying PID Controller to Inverted Pendulum

Have you ever balanced a pencil on your finger? Everyone knows that's hard to do. However, with a PID controller, that might be possible!

In this part, we will walk through how you will keep an inverted pendulum upright using the PID controller we just learned. This notebook will serve as a simulation, but you can build a system like this in real life. Check out [this demo from EE 128](https://www.youtube.com/watch?v=6MEfrPqvptA). In this case, every time the pendulum is hit (aka the system state changes), we can apply control signals to make it upright again! That's how powerful controls are.

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('6MEfrPqvptA', width=800, height=300)

# Watch the demo below!

First, we need to import some libraries to help us with the calculation. 
Throughout your time at Berkeley, you will be very familiar with `numpy` for doing mathematical operations (particular matrixes), `time` for timing and time-stepping, `matplotlib` for plotting. 

It is worth noting that many control problems involve significant amounts of linear algebra and matrix operations, and `numpy` is an excellent library for performing matrix operations easily and fast.

In [ ]:
import numpy as np, math, time, matplotlib.pyplot as plt, sys
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

Here we imagine the system composes of a pendulum on top of a cart. The pendulum is unstable (not upright and is falling), so we must move the cart to keep it stable and balanced. To do so, we can apply a force on the cart at each timestep.

Note that we are creating classes for these objects. Object-oriented programming is essential in simulations and representing the physical world. We can create many instances of these objects with these classes, each with associated data representing its physical properties.

We mentioned **state** represents the current physical description of the system. In this case, we focus on the pendulum, and the states we account for are its length, theta, and ball_mass. The only state that can change is theta, the angle between the pendulum from its upright position. To keep it upright, we must make theta go to zero.

**Our goal**: Apply force on the cart, so theta **reaches** and **stabilizes** at zero (upright position!)

In [ ]:
class Cart:
    def __init__(self,x,mass):
        self.x = x  
        self.y = 0
        self.mass = mass

class Pendulum:
    def __init__(self,length,theta,ball_mass):
        self.length = length
        self.theta = theta
        self.ball_mass = ball_mass

Let's initiate some constants for simulation. It is always good to put them at the start of a simulation and declare them as global variables. Here we only need to consider the gravitational constant g. What are some other factors that we can account for in a more complex situation?

In [ ]:
#Constants:
g = 9.81

Here is our **PID Controller** in action! Recall the equation for our controller:
$$u(t)=K_{\text{p}}e(t)+K_{\text{i}}\int _{0}^{t}e(t')\,dt'+K_{\text{d}}{\frac {de(t)}{dt}}$$

We are giving the controller our $e(t)$, the difference between the desired state and the current state.
We have the gains (the three $K$ constants) tuned and passed in here. With this information, we can amplify the effect of proportional, integral, and derivative operations on the $e(t)$ by the three $K$ constants, respectively. The combined effect is our control signal $u(t)$!

This controller now gives us the input to the actuator/force that we would like the actuator to perform! We will keep using the controller to calculate the next control signal, based on every new state of our $e(t)$. Hopefully, we will make the pendulum upright over time!

In [ ]:
def PID_Control(cart, pendulum, dt, error, previous_error, integral, Kp=-150, Kd=-20, Ki=-20):
    # Using PID to find control inputs

    # The gains were emperically tuned

    derivative = (error - previous_error) / dt
    integral += error * dt
    Output_force = (Kp * error) + (Kd * derivative) + (Ki * integral)
    return Output_force, integral

Here is some code translating the force into physical effect on the cart position. It also determines the pendulum's angle/orientation after the output force is applied to the system by the actuator. Remember that our PID controller calculated the force/control signal! After we apply this force, our state of the pendulum changes, `pendulum.theta`, so we can use this new state to feed into the PID controller again in the next timestep.

In [ ]:
#These apply a force F using the equations of motion for the cart and pendulum.

def Apply_Force(cart, pendulum, F, dtime, x_2, dtheta, theta_2, previous_dtime):
    
    a_theta = (((cart.mass + pendulum.ball_mass) * g * math.sin(pendulum.theta)) + (F * math.cos(pendulum.theta)) - \
        (pendulum.ball_mass * ((dtheta)**2.0) * pendulum.length * math.sin(pendulum.theta) * math.cos(pendulum.theta))) \
         / (pendulum.length * (cart.mass + (pendulum.ball_mass * (math.sin(pendulum.theta)**2.0)))) 
    
    a_x = ((pendulum.ball_mass * g * math.sin(pendulum.theta) * math.cos(pendulum.theta)) - \
        (pendulum.ball_mass * pendulum.length * math.sin(pendulum.theta) * (dtheta**2)) + (F)) \
         / (cart.mass + (pendulum.ball_mass * (math.sin(pendulum.theta)**2)))
    
    cart.x += ((dtime**2) * a_x) + (((cart.x - x_2) * dtime) / previous_dtime)
    
    pendulum.theta += ((dtime**2)*a_theta) + (((pendulum.theta - theta_2)*dtime) / previous_dtime)


Here is our implementation of $e(t)$, the difference between the current state (theta of the pendulum) and the desired state. Remember, we want our pendulum to be upright, so the desired theta will be 0 as theta is the angular difference between the pendulum and the upright position. 

In [ ]:
def find_error(pendulum):

    # This function returns the error
    error = (pendulum.theta % (2 * math.pi)) - 0
    if error > math.pi:
        error -= (2 * math.pi)
    return error

Helper function for visualization. We would like to see the following information.
* **errors**, or $e(t)$, how far away the pendulum is from upright position 
* **theta**, the state of the pendulum, which is the angular position of the pendulum.
* **force**, or $u(t)$, the output from our PID controller
* **x**, or `cart.x`, the location of the cart on the x-axis

In [ ]:
def plot_graphs(times,errors,theta,force,x):
    # This function plots all the graphs
    plt.figure(figsize=(20,10))
    
    plt.subplot(4, 1, 1)
    plt.plot(times,errors,'-b')
    plt.ylabel('Error')
    plt.xlabel('Time')

    plt.subplot(4, 1, 2)
    plt.plot(times,theta,'-b')
    plt.ylabel('Theta')
    plt.xlabel('Time')

    plt.subplot(4, 1, 3)
    plt.plot(times,force,'-b')
    plt.ylabel('Force')
    plt.xlabel('Time')

    plt.subplot(4, 1, 4)
    plt.plot(times,x,'-b')
    plt.ylabel('X')
    plt.xlabel('Time')

    plt.show()

Putting all components together to build a simulation of our controller trying to balance the Pendulum, over a period of time.

In [ ]:
#Making it a function so we can reuse it and use it wrapper functions

def runSim(mass_of_cart=5, mass_of_ball=1, Kp=-150, Kd=-20, Ki=-20):
    
    print("Simulating with mass_of_cart={0}, mass_of_ball={1}, Kp={2}, Kd={3}, Ki={4}...".format(mass_of_cart, mass_of_ball, Kp, Kd, Ki))
    
    # instantiates cart and pendulum objects
    cart = Cart(0, mass_of_cart)
    pendulum = Pendulum(1, 0.3, mass_of_ball)
    
    # Record data for plotting
    errors, force, theta, times, x = [],[],[],[],[]
    simulation_time = 5 # if you have more memory, feel free to increase this
    previous_timestamp = time.time()
    start_time = previous_timestamp
    end_time = previous_timestamp + simulation_time
    
    d_theta = 0
    theta_1 = theta_2 = pendulum.theta # x_{t-1} = x_{t-2} = pendulum starting point
    x_1 = x_2 = cart.x
    previous_error = find_error(pendulum)
    integral = 0
    previous_dtime = 0
    
    while time.time() <= end_time:
    
        #calculating timestep (dtime) and the current error.
        current_timestamp = time.time()
        dtime = (current_timestamp - previous_timestamp)
        error = find_error(pendulum)
    
        if previous_dtime != 0: # This condition is to make sure that theta_dot is not infinity in the first step
        
        
            #finding the derivatives of the angle and x position over the timestep.
            d_theta = (theta_1 - theta_2 ) / previous_dtime
            d_x = (x_1 - x_2) / previous_dtime
        
            #finding the control and applying it to the card.
            F,integral = PID_Control(cart, pendulum, dtime, error, previous_error, integral, Kp=Kp, Kd=Kd, Ki=Ki)
            Apply_Force(cart, pendulum, F, dtime, x_2, d_theta, theta_2, previous_dtime)
        
            # For plotting the graphs
        
            force.append(F)            
            x.append(cart.x)
            errors.append(error)
            times.append(current_timestamp - start_time)
            theta.append(pendulum.theta)
        
        # Update the variables
        
        previous_dtime = dtime
        previous_timestamp = current_timestamp
        previous_error = error
    
        theta_2 = theta_1
        theta_1 = pendulum.theta
        x_2 = x_1
        x_1 = cart.x

    plot_graphs(times,errors,theta,force,x)
    print("Done.")

### PID Controller in Action

We have built the simulation so let's now run it! See how overtime, the PID controller is applying forces (see *Force*) based on the state (*Theta* and *Error*), and eventually bringing the error to 0, meaning we have successfully kept our Pendulum upright!

Try the following!
1. Observe the graph. Do you see some oscillations? Do we sometimes overcorrect? Do you notice how closed-loop feedback control can eventually bring everything to stability in the end?
2. Adjust the `mass_of_cart` and `mass_of_ball`. See how changing the physical properties of the simulation changes the graph.
3. Adjust the `Kp`, `Kd`, `Ki`. We know these are constants (or gains) that amplify proportional, integral, and derivative operations, whose weighted net effect generates the force applied. These constants define each controller, so try to change them and see how the system responds. Are you able to create a controller that brings the system to stability faster?

<font color='red'>**Warning:**</font> The notebook might take some time to recalculate and plot every time you move the slider. This notebook also suffers from intensive memory usage. Thus after a few slider adjustments, we recommend you restart kernel and run all to release some of the memories used and then continue to try new values. Alternatively, you can manually input the parameter into the argument of the `interact` function.


In [ ]:
interact(runSim, mass_of_cart=widgets.IntSlider(min=1, max=10, step=1, value=5), \
                 mass_of_ball=widgets.IntSlider(min=1, max=10, step=1, value=1), \
                 Kp=widgets.IntSlider(min=-200, max=200, step=5, value=-150), \
                 Kd=widgets.IntSlider(min=-200, max=200, step=5, value=-20), \
                 Ki=widgets.IntSlider(min=-200, max=200, step=5, value=-20))

## Congratulations! 
You now know what control is and how a PID controller works! Note, PID is only one approach, where we use proportional, integral, derivative operations on $e(t)$. Can you come up with a different controller? What are the pros and cons of each kind of controller? How can you decide when to use which one? How do you determine the gains/tuning constants/ $K$? These are all things that you can explore throughout your journey at Berkeley EECS.

To summarize this lab, the key takeaways are:
1. Embedded Systems sense information and perform actions.
2. The real world is complicated, and when we perform actions, we might not reach the goal at one go. We need constant adjustment to get to and stabilize at the desired state.
3. We can design a closed-loop feedback system, where the signal to the actuator is dependant on the current state of the system. In other words, the output of the controller depends on the previous input into the controller (the state). 
4. We learned about how a PID controller works. It uses proportional, integral, and derivative operations to create a control signal. We scale each operation by applying a gain/tuning constant/ $K$ / weight. These $K$ values are characteristics of our PID controller.
5. We simulated this controller by building a physics engine and evaluated its performance. We tried to improve its performance by adjusting the $K$ values (aka updating our controller design). 

## Resources
If you find this interesting, check out these courses you can take at Berkeley!

* EECS 16B - Designing Information Devices and Systems II
* EE C106A - Introduction to Robotics
* EE C106B - Robotic Manipulation and Interaction
* EE 120 - Signals and Systems
* EECS 126 - Probability and Random Processes
* EE C128 - Feedback Controls
* EE 221A - Linear Systems
* EE 222 - Nonlinear Systems

Controls are incredibly important and useful for building embedded systems, robots, and machine learning applications. This is definitely something worth learning.

## References

Thanks to the work of these folks!
* https://github.com/Nikkhil16/Inverted_Pendulum,
* https://sites.google.com/site/fpgaandco/pid
* https://www.moorepants.info/blog/npendulum.html
* https://github.com/gilbertgede/idetc-2013-paper/blob/master/n-pendulum-control.ipynb,